<a href="https://colab.research.google.com/github/trvoid/llm-study/blob/main/bert/getting_started_with_distilbert_for_qa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DistilBERT for MLM 시작하기 (한국어)

**노트: distilbert-base-multilingual-cased를 기본 모델로 사용하였음**

이 실습은 아래 문서의 내용을 토대로 진행하였습니다.

* [2. 마스크 언어 모델(Masked Language Model) 미세조정](https://wikidocs.net/166833), Transformers (신경망 언어모델 라이브러리) 강좌
* [Fine-tuning a masked language model](https://huggingface.co/learn/llm-course/en/chapter7/3), LLM Course in Hugging Face

사용할 모델과 데이터셋은 다음과 같습니다.

* [DistilBERT](distilbert/distilbert-base-multilingual-cased):기본 모델 (단어에 대한 단순 임베딩이 아니라 맥락을 고려한 임베딩 수행)
* [DistilBertForMaskedLM](https://huggingface.co/docs/transformers/model_doc/distilbert#transformers.DistilBertForMaskedLM): DistilBERT 모델에 마스크 언어 모델 층을 추가한 것으로서 마스크 언어 모델 미세조정 훈련을 위한 모델
* [KorQuAD 1.0](https://huggingface.co/datasets/KorQuAD/squad_kor_v1): 질의응답 미세조정 훈련을 위한 데이터셋

## 토크나이저 및 모델 적재

In [1]:
import torch

# GPU 사용 가능 여부 확인 및 장치 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

device: cuda


In [2]:
from transformers import AutoTokenizer, DistilBertForMaskedLM

model_checkpoint = "distilbert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = DistilBertForMaskedLM.from_pretrained(model_checkpoint).to(device)

2025-04-09 16:25:43.972208: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-04-09 16:25:43.980482: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1744183543.989090 2943846 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1744183543.991702 2943846 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1744183543.999729 2943846 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [3]:
print(f"vocab size: {len(tokenizer)}")
print(f"model device: {next(model.parameters()).device}")
distilbert_num_parameters = model.num_parameters() / 1_000_000
print(f"DistilBERT number of parameters: {round(distilbert_num_parameters)}M")
print(f"BERT number of parameters: 110M")

vocab size: 119547
model device: cuda:0
DistilBERT number of parameters: 135M
BERT number of parameters: 110M


In [4]:
def test_tokenizer(tokenizer, text):
    inputs = tokenizer(text, return_tensors="pt", max_length=14, padding="max_length")
    print(f"input_ids: {inputs["input_ids"][0]}")
    print(f"len(input_ids): {len(inputs["input_ids"][0])}")
    
    token_strs = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])
    print(f"token_strs: {token_strs}")
    
    decoded_text = tokenizer.decode(inputs["input_ids"][0])
    print(f"decoded_text: {decoded_text}")

org_text = "미세먼지가 심하면 차량 2부제와 같은 비상저감조치를 시행"
text = "미세먼지가 심하면 차량 2부제와 [MASK] 비상저감조치를 시행"

test_tokenizer(tokenizer, org_text)
test_tokenizer(tokenizer, text)

input_ids: tensor([   101,   9309,  24982, 118922,  80795,   9491,  38378,   9730,  44321,
           123,  14646,  17730,  12638,  18589,   9379,  14871,  48387, 105197,
         20626,  62672,   9485,  25549,    102])
len(input_ids): 23
token_strs: ['[CLS]', '미', '##세', '##먼', '##지가', '심', '##하면', '차', '##량', '2', '##부', '##제', '##와', '같은', '비', '##상', '##저', '##감', '##조', '##치를', '시', '##행', '[SEP]']
decoded_text: [CLS] 미세먼지가 심하면 차량 2부제와 같은 비상저감조치를 시행 [SEP]
input_ids: tensor([   101,   9309,  24982, 118922,  80795,   9491,  38378,   9730,  44321,
           123,  14646,  17730,  12638,    103,   9379,  14871,  48387, 105197,
         20626,  62672,   9485,  25549,    102])
len(input_ids): 23
token_strs: ['[CLS]', '미', '##세', '##먼', '##지가', '심', '##하면', '차', '##량', '2', '##부', '##제', '##와', '[MASK]', '비', '##상', '##저', '##감', '##조', '##치를', '시', '##행', '[SEP]']
decoded_text: [CLS] 미세먼지가 심하면 차량 2부제와 [MASK] 비상저감조치를 시행 [SEP]


In [5]:
import torch

def find_topk_for_masked(tokenizer, model, text, topk=5):
    inputs = tokenizer(text, return_tensors="pt")
    inputs = {k: v.to(device) for k, v in inputs.items() if isinstance(v, torch.Tensor)}

    token_logits = model(**inputs).logits
    #print(token_logits.shape)
    
    # [MASK]의 위치를 찾고, 해당 logits을 추출합니다.
    #print(torch.where(inputs["input_ids"] == tokenizer.mask_token_id))
    mask_token_index = torch.where(inputs["input_ids"] == tokenizer.mask_token_id)[1]
    #print(mask_token_index)
    mask_token_logits = token_logits[0, mask_token_index, :]
    #print(mask_token_logits)
    
    # 가장 큰 logits값을 가지는 [MASK] 후보를 선택합니다.
    top_5_tokens = torch.topk(mask_token_logits, topk, dim=1).indices[0].tolist()
    
    return top_5_tokens

topk_tokens = find_topk_for_masked(tokenizer, model, text, 5)
for token in topk_tokens:
    print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

'>>> 미세먼지가 심하면 차량 2부제와 ##의 비상저감조치를 시행'
'>>> 미세먼지가 심하면 차량 2부제와 ##가 비상저감조치를 시행'
'>>> 미세먼지가 심하면 차량 2부제와 ##이 비상저감조치를 시행'
'>>> 미세먼지가 심하면 차량 2부제와 ##와 비상저감조치를 시행'
'>>> 미세먼지가 심하면 차량 2부제와 ##지 비상저감조치를 시행'


## 데이터셋 적재

In [6]:
from datasets import load_dataset

dataset_name = "KorQuAD/squad_kor_v1"
dataset = load_dataset(dataset_name, trust_remote_code=True)
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 60407
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 5774
    })
})


In [7]:
samples = dataset["train"].shuffle(seed=42).select(range(3))

for row in samples:
    print(f"\n'>>> context: {row['context']}'")
    print(f"'>>> question: {row['question']}'")


'>>> context: 9월 26일 환경부를 비롯한 12개 관계부처가 합동으로 '미세먼지 관리 종합대책'을 확정하고 발전·산업·수송·생활 등 4개 부분에서 저감 대책을 실시하는 관련 로드맵을 발표했다. 7조 2000억 원의 예산을 투입해 미세먼지 국내 배출량을 30% 감축하고 미세먼지 '나쁨' 일수를 70%까지 줄이기로 한 것인데 이를 위해 공정률 10% 미만인 석탄발전소 9기 중 4기를 LNG 등 친환경 연료로 전환하고 남은 5기도 최고 수준의 배출 기준을 적용하며 30년이 넘은 노후 석탄발전소 7기는 임기 내 폐쇄하기로 했다. 또한 대기배출총량제를 전국으로 확대·강화하고 먼지총량제를 새로 도입하며, 노후 경유차 221만 대를 임기 내 77% 조기 폐차하고 친환경 차를 2022년까지 200만 대 보급하며 미세먼지가 심하면 차량 2부제와 같은 비상저감조치를 시행하기로 했다. 국제적으로는 미세먼지를 한중 양국의 정상의제로 격상하고 동북아 지역에서 협약체결을 추진하면서 미세먼지 환경기준도 선진국 수준으로 강화할 것도 포함했다.'
'>>> question: 미세먼지 해결을 위해 전국으로 확대 강화된 기존의 제도는?'

'>>> context: 프리스틀리는 워링턴 거주 시절에는 다른 일 때문에 신학 연구에 몰두하지 못하였으나, 리즈에 오면서 그는 신학 연구에 많은 시간을 투자하였고, 결과적으로 그의 신앙은 아리우스주의에서 유니테리언으로 정립되었다. 리즈에서 프리스틀리는 삼위일체와 예수의 선재성(先在性, 성자인 예수는 천지창조전부터 성부와 같이 존재했다는 교리)등을 완전히 부정하였고, 기독교 교리와 성서를 새롭게 해석하기 시작했다. 그는 오래전부터 써오던 신학 교육에 대한 책인 《자연과 계시 종교의 원리》(Institutes of Natural and Revealed Religion)를 출판하기 시작하였는데, 1772년에 1권이 출판되었고 마지막 3권은 1774년에 출판되었다. 그는 책에서 자연 종교, 계시의 진실성을 뒷받침하는 논거, 계시로부터 얻을 수 있는 진실

## 훈련 데이터 준비

In [8]:
def tokenize_function(examples, max_length=512):
    result = tokenizer(examples["question"], 
                       examples["context"],
                       max_length=max_length, 
                       truncation="only_second"
                      )
    if tokenizer.is_fast:
        result["word_ids"] = [result.word_ids(i) for i in range(len(result["input_ids"]))]
    return result

In [9]:
test_examples = {
    "context": ["고양이가 의자에 앉습니다."],
    "question": ["고양이가 어디에 앉습니까?"],
    "answers": [
        {
            "text": ["의자"],
            "answer_start": [5]
        }
    ]
}
test_result = tokenize_function(test_examples)
print(test_result)
token_strs = tokenizer.convert_ids_to_tokens(test_result["input_ids"][0])
print(token_strs)

{'input_ids': [[101, 8888, 37114, 57362, 9546, 48446, 10530, 9522, 119081, 25503, 118671, 136, 102, 8888, 37114, 57362, 9637, 13764, 10530, 9522, 119081, 48345, 119, 102]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'word_ids': [[None, 0, 0, 0, 1, 1, 1, 2, 2, 2, 2, 3, None, 0, 0, 0, 1, 1, 1, 2, 2, 2, 3, None]]}
['[CLS]', '고', '##양', '##이가', '어', '##디', '##에', '앉', '##습', '##니', '##까', '?', '[SEP]', '고', '##양', '##이가', '의', '##자', '##에', '앉', '##습', '##니다', '.', '[SEP]']


In [10]:
# 빠른 멀티스레딩을 작동시키기 위해서, batched=True를 지정합니다.
tokenized_datasets = dataset.map(
    tokenize_function, batched=True, remove_columns=dataset["train"].column_names
)
print(tokenized_datasets)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 60407
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids'],
        num_rows: 5774
    })
})


In [11]:
print(f"model_max_length: {tokenizer.model_max_length}")

model_max_length: 512


In [12]:
#chunk_size = 128
chunk_size = 150

In [13]:
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_datasets["train"][:3]

for idx, sample in enumerate(tokenized_samples["input_ids"]):
    print(f"Review {idx} length: {len(sample)}")

Review 0 length: 429
Review 1 length: 428
Review 2 length: 428


In [14]:
concatenated_examples = {
    k: sum(tokenized_samples[k], []) for k in tokenized_samples.keys()
}
total_length = len(concatenated_examples["input_ids"])
print(f"Concatenated reviews length: {total_length}")

Concatenated reviews length: 1285


In [15]:
chunks = {
    k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
    for k, t in concatenated_examples.items()
}

for chunk in chunks["input_ids"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 150'
'>>> Chunk length: 150'
'>>> Chunk length: 150'
'>>> Chunk length: 150'
'>>> Chunk length: 150'
'>>> Chunk length: 150'
'>>> Chunk length: 150'
'>>> Chunk length: 150'
'>>> Chunk length: 85'


In [16]:
def group_texts(examples):
    # 모든 텍스트들을 결합한다.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    # 결합된 텍스트들에 대한 길이를 구한다.
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # `chunk_size`보다 작은 경우 마지막 청크를 삭제
    total_length = (total_length // chunk_size) * chunk_size
    # max_len 길이를 가지는 chunk 단위로 슬라이스
    result = {
        k: [t[i : i + chunk_size] for i in range(0, total_length, chunk_size)]
        for k, t in concatenated_examples.items()
    }
    # 새로운 레이블 컬럼을 생성
    result["labels"] = result["input_ids"].copy()
    return result                            

In [17]:
lm_datasets = tokenized_datasets.map(group_texts, batched=True)
print(lm_datasets)

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 132044
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 12771
    })
})


In [18]:
tokenizer.decode(lm_datasets["train"][1]["input_ids"])

'교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡 ('

In [19]:
tokenizer.decode(lm_datasets["train"][1]["labels"])

'교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡 ('

In [20]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)

In [21]:
samples = [lm_datasets["train"][i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")

for chunk in data_collator(samples)["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] 바그너는 괴 [MASK]의 파우스트를 [MASK]고 무엇을 쓰고자 했는 [MASK]? [SEP] 1839년 바 [MASK]너는 괴테의 파우스트을 처음 읽고 [MASK] 내용에 마음 règle 끌려 이를 [MASK]재 [MASK] 해서 [MASK] 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바 [MASK]너는 1838년에 빛 독촉으로 산전수전을 다 [UNK] 상황이라 좌절과 실망에 가득했으며 메피스토펠 [MASK]를 만나는 파우스트 [MASK] 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관 [MASK]악단이 연주하는 베토벤의'

'>>> 교향 [MASK] [MASK]번을 [MASK]고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파 [MASK]트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리 [MASK] 것은 의심 [MASK] 여지가 없다. 여기의 라 [MASK]조 조성의 경우에도 그의 전기에 적혀 있는 [MASK]처럼 단순한 [MASK]신 [MASK] 피로나 실의가 [MASK]영된 것이 [MASK] 베토벤 [MASK] 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 [MASK] 파리에서 착수했으나 [MASK]악 [MASK] 쓴 뒤에 중단했다. 또한 작 [MASK]의 [MASK]성과 동시에 그는 이 서곡 ('


In [22]:
import collections
import numpy as np

from transformers import default_data_collator

wwm_probability = 0.2

def whole_word_masking_data_collator(features):
    for feature in features:
        word_ids = feature.pop("word_ids")

        # 단어와 해당 토큰 인덱스 간의 map 생성
        mapping = collections.defaultdict(list)
        current_word_index = -1
        current_word = None
        for idx, word_id in enumerate(word_ids):
            if word_id is not None:
                if word_id != current_word:
                    current_word = word_id
                    current_word_index += 1
                mapping[current_word_index].append(idx)

        # 무작위로 단어 마스킹
        mask = np.random.binomial(1, wwm_probability, (len(mapping),))
        input_ids = feature["input_ids"]
        labels = feature["labels"]
        new_labels = [-100] * len(labels)
        for word_id in np.where(mask)[0]:
            word_id = word_id.item()
            for idx in mapping[word_id]:
                new_labels[idx] = labels[idx]
                input_ids[idx] = tokenizer.mask_token_id

    return default_data_collator(features)

In [23]:
samples = [lm_datasets["train"][i] for i in range(2)]
batch = whole_word_masking_data_collator(samples)

for chunk in batch["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] [MASK] [MASK] [MASK] [MASK] 괴테의 파우스트를 읽고 무엇을 쓰고자 했는가? [SEP] 1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다 [MASK] 이 시기 바그너는 [MASK] [MASK] 빛 독촉으로 산전수전을 다 [UNK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] 실망에 [MASK] [MASK] [MASK] 메피스토펠레스를 만나는 [MASK] [MASK] [MASK] [MASK] 심경에 공감했다고 한다 [MASK] 또한 파리에서 아브네크의 지휘로 파리 [MASK] [MASK] 관현악단이 연주하는 베토벤의'

'>>> 교향곡 9번을 [MASK] [MASK] 깊은 [MASK] [MASK] 받았는데, 이것이 이듬해 1월에 파우스트의 [MASK] [MASK] [MASK] 쓰여진 이 작품에 조금이라도 [MASK] [MASK] [MASK] [MASK] [MASK] [MASK] 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 [MASK] 것처럼 단순한 정신적 피로나 실의가 반영된 [MASK] [MASK] 베토벤의 합창교향곡 조성의 [MASK] 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 [MASK] [MASK] [MASK] 쓴 [MASK] [MASK] 중단했다 [MASK] 또한 작품의 완성과 동시에 그는 이 서곡 [MASK]'


훈련 데이터 개수와 테스트 데이터 개수를 지정하여 샘플링합니다.

In [24]:
#train_size = 10_000
#test_size = int(0.1 * train_size)
train_size = None
test_size = 0.1

downsampled_dataset = lm_datasets["train"].train_test_split(
    train_size=train_size, test_size=test_size, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 118839
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 13205
    })
})

## 미세조정 훈련

In [25]:
from transformers import TrainingArguments, Trainer

batch_size = 64
# Show the training loss with every epoch
logging_steps = len(downsampled_dataset["train"]) // batch_size
model_name = model_checkpoint.split("/")[-1]

training_args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-korquad",
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    num_train_epochs=4.0,
    learning_rate=2e-5,
    weight_decay=0.01,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=False,
    fp16=True,
    logging_steps=logging_steps,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=downsampled_dataset["train"],
    eval_dataset=downsampled_dataset["test"],
    data_collator=data_collator,
)

/home/wjeong/DevEnv/py312/lib/python3.12/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [26]:
import math

eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity: 67.94


In [27]:
trainer.train()

finetuned_model_path = "./fine-tuned-distilbert-korquad-mlm"
tokenizer.save_pretrained(finetuned_model_path)
model.save_pretrained(finetuned_model_path)

Epoch,Training Loss,Validation Loss,Model Preparation Time
1,2.599800,2.192273,0.000500
2,2.258800,2.054537,0.000500
3,2.149700,1.996299,0.000500
4,2.103300,1.967951,0.000500


In [28]:
eval_results = trainer.evaluate()
print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

>>> Perplexity: 7.11


## 모델 사용

In [29]:
test_texts = [
    "미세먼지가 심하면 차량 2부제와 [MASK] 비상저감조치를 시행", 
    "미[MASK]먼지가 심하면 차량 2부제와 같은 비상저감조치를 시행",
    "미세먼지가 심[MASK] 차량 2부제와 같은 비상저감조치를 시행"
]
for text in test_texts:
    print(f"'input text: {text}'")
    topk_tokens = find_topk_for_masked(tokenizer, model, text, topk=5)
    for token in topk_tokens:
        print(f"'>>> {text.replace(tokenizer.mask_token, tokenizer.decode([token]))}'")

'input text: 미세먼지가 심하면 차량 2부제와 [MASK] 비상저감조치를 시행'
'>>> 미세먼지가 심하면 차량 2부제와 함께 비상저감조치를 시행'
'>>> 미세먼지가 심하면 차량 2부제와 동시에 비상저감조치를 시행'
'>>> 미세먼지가 심하면 차량 2부제와 같은 비상저감조치를 시행'
'>>> 미세먼지가 심하면 차량 2부제와 같이 비상저감조치를 시행'
'>>> 미세먼지가 심하면 차량 2부제와 달리 비상저감조치를 시행'
'input text: 미[MASK]먼지가 심하면 차량 2부제와 같은 비상저감조치를 시행'
'>>> 미##리먼지가 심하면 차량 2부제와 같은 비상저감조치를 시행'
'>>> 미##치는먼지가 심하면 차량 2부제와 같은 비상저감조치를 시행'
'>>> 미##치먼지가 심하면 차량 2부제와 같은 비상저감조치를 시행'
'>>> 미##의먼지가 심하면 차량 2부제와 같은 비상저감조치를 시행'
'>>> 미##터먼지가 심하면 차량 2부제와 같은 비상저감조치를 시행'
'input text: 미세먼지가 심[MASK] 차량 2부제와 같은 비상저감조치를 시행'
'>>> 미세먼지가 심##한 차량 2부제와 같은 비상저감조치를 시행'
'>>> 미세먼지가 심##해 차량 2부제와 같은 비상저감조치를 시행'
'>>> 미세먼지가 심##각 차량 2부제와 같은 비상저감조치를 시행'
'>>> 미세먼지가 심##하게 차량 2부제와 같은 비상저감조치를 시행'
'>>> 미세먼지가 심##리 차량 2부제와 같은 비상저감조치를 시행'
